# Segmenting and Clustering Neighborhoods in Toronto

This notebook contains all three sections. There are headers to indicate the start of each question. 

## Part 1

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Scrapping the table from the website

In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
hood=pd.DataFrame(df[0])
print(hood.shape)
hood.head()

(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Cleaning the data - Dropping rows where Borough is Not assigned

In [3]:
hood = hood[hood['Borough']!='Not assigned']
hood.reset_index(drop=True, inplace = True)
print(hood.shape)
hood.head()

(103, 3)


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


Checking if any Neighborhoods are not assigned

In [4]:
msk = hood['Neighborhood'] == ('NaN' or 'Not assigned')

if sum(msk) == 0: 
    print('All are assinged')



All are assinged


Checking for multiple entries for each post code

In [5]:
pc=hood['Postal code'].unique()

if pc.shape[0] == hood.shape[0]:
    print('no postal codes have multiple entries')


no postal codes have multiple entries


### Part 1 Answer

In [6]:
hood.shape

(103, 3)

# Part 2

In [7]:
import requests
import numpy as np

In [8]:
#!conda install -c conda-forge geocoder --yes
import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Finding the Lat Long for each borough

In [9]:
coords= pd.read_csv('Geospatial_Coordinates.csv')
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
L=hood.shape[0]
Lat=np.zeros(L)
Lon=np.zeros(L)

for i in range(L):
    lat_lng_coords = None
    postal_code=hood['Postal code'][i]
    LL= coords[coords['Postal Code'] == postal_code]
    Lat[i] = LL['Latitude'].values
    Lon[i] = LL['Longitude'].values  


In [11]:
hood['Latitude']=Lat
hood['Longitude']=Lon
hood.tail(10)

,Postal code,Borough,Neighborhood,Latitude,Longitude
93,M8W,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
94,M9W,Etobicoke,Northwest,43.706748,-79.594054
95,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
96,M4X,Downtown Toronto,St. James Town / Cabbagetown,43.667967,-79.367675
97,M5X,Downtown Toronto,First Canadian Place / Underground city,43.648429,-79.382280
98,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509
102,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,43.628841,-79.520999


# Part 3

Lets explore Toronto... First where are all these neighborhoods

In [12]:
print('there are ', len(hood['Borough'].unique()),' boroughs and ', len(hood['Neighborhood']), 'neighborhoods' )

there are  10  boroughs and  103 neighborhoods


Lets Map them

In [13]:
#set the latLog for the plot center to be the mean of the lat long of the neigborhoods
lat = hood['Latitude'].mean()
long = hood['Longitude'].mean()
print('mean coordinates are: ', lat, long)

mean coordinates are:  43.70460773398059 -79.39715291165048


In [14]:
import folium 
map_tor= folium.Map(location=[lat, long], zoom_start=11)

for lt, lng, borough, neighborhood in zip(hood['Latitude'], hood['Longitude'], hood['Borough'], hood['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lt, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)      


map_tor

### Now lets learn about them - query Foursquares

In [27]:

CLIENT_ID = 'AVXRZRL3A01BX2ZCDFJH0RE4UPDJIH4NWBGIHXSQUNRUVQAL' # your Foursquare ID
CLIENT_SECRET = 'MWUCP1EXZY22ZCBDXUSQOVPYDF4ZYP22JB3ANVGBN4OWZOO4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Defining credentials:')
print('CLIENT_ID: ' + CLIENT_ID)


Defining credentials:
CLIENT_ID: AVXRZRL3A01BX2ZCDFJH0RE4UPDJIH4NWBGIHXSQUNRUVQAL


Using the functions from the Neigborhoods Lab - defining them here: 

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Looking for nearby venues for each neighborhood.

In [40]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius in meters

print('Collecting information on:')
Venues = getNearbyVenues(names=hood['Neighborhood'],
                                   latitudes=hood['Latitude'],
                                   longitudes=hood['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

In [41]:
print('A total of ', Venues.shape[0], 'venues were found. There are ', len(Venues['Venue Category'].unique()) , ' unigue catergories') 
Venues.head()

A total of  2192 venues were found. There are  270  unigue catergories


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


### Lets analyze them - format, cluster, and plot

Create dummy values for venue types

In [42]:
# one hot encoding
Venues_onehot = pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep="")
Venues_onehot.drop(['Neighborhood'], axis = 1, inplace = True)

# add neighborhood column back to dataframe
Venues_onehot.insert(0, 'Neighborhood', Venues['Neighborhood']) 

Venues_onehot.head()



,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by neighborhood

In [43]:
Venues_grouped = Venues_onehot.groupby('Neighborhood').mean().reset_index()
Venues_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
1,Alderwood / Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
2,Bathurst Manor / Wilson Heights / Downsview North,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.045455,0.000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
4,Bedford Park / Lawrence Manor East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Willowdale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.025,0.0,0.0,0.0,0.0,0.0
90,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
91,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0
92,York Mills / Silver Hills,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000,0.0,0.0,0.0,0.0,0.0


Lets find the 10 most popular venues in each and write that into a new data frame, using the code from the lab

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
hoods_Venues_sorted = pd.DataFrame(columns=columns)
hoods_Venues_sorted['Neighborhood'] = Venues_grouped['Neighborhood']

for ind in np.arange(Venues_grouped.shape[0]):
       hoods_Venues_sorted.iloc[ind, 1:] = return_most_common_venues(Venues_grouped.iloc[ind, :], num_top_venues)

hoods_Venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Skating Rink,Breakfast Spot,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,...,Drugstore,Donut Shop,Ethiopian Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Dessert Shop,Department Store,Deli / Bodega,Dance Studio
1,Alderwood / Long Branch,Pizza Place,Coffee Shop,Pharmacy,Pub,Pool,Sandwich Place,Dance Studio,Gym,Skating Rink,...,Doner Restaurant,Distribution Center,Curling Ice,Diner,Donut Shop,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Discount Store
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Gift Shop,Frozen Yogurt Shop,Sandwich Place,Bridal Shop,Diner,Restaurant,Deli / Bodega,...,Ice Cream Shop,Sushi Restaurant,Middle Eastern Restaurant,Fried Chicken Joint,Shopping Mall,Pharmacy,Grocery Store,Gas Station,Video Store,Pizza Place
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,...,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Dim Sum Restaurant,Deli / Bodega,Department Store
4,Bedford Park / Lawrence Manor East,Restaurant,Coffee Shop,Sandwich Place,Italian Restaurant,Pharmacy,Indian Restaurant,Pub,Sushi Restaurant,Café,...,Thai Restaurant,Pizza Place,Butcher,Juice Bar,American Restaurant,Grocery Store,Greek Restaurant,Comfort Food Restaurant,Liquor Store,Discount Store


Now cluster the neigborhoods using K-means

In [46]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 6

Venues_grouped_clustering = Venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [47]:
# add clustering labels
hoods_Venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hood_merged = hood

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hood_merged = hood_merged.join(hoods_Venues_sorted.set_index('Neighborhood'), on='Neighborhood')

hood_merged.head(6) 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Dog Run,Dessert Shop,...,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Coffee Shop,Pizza Place,Intersection,...,Dog Run,Distribution Center,Deli / Bodega,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Department Store,Cupcake Shop
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,...,Beer Store,Spa,Farmers Market,Shoe Store,Yoga Studio,Electronics Store,Distribution Center,Dessert Shop,Chocolate Shop,Performing Arts Venue
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,...,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Diner,Music Venue,Beer Bar,...,College Auditorium,Creperie,Bank,Mexican Restaurant,Discount Store,Distribution Center,Juice Bar,Italian Restaurant,Fried Chicken Joint,Hobby Shop
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
#some neighborhoods had not results from Foursquares, assigning them to group thier own cluster
kclusters= kclusters + 1
hood_merged['Cluster Labels'].replace(np.nan, kclusters, inplace = True)

hood_merged.head(6)


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Dog Run,Dessert Shop,...,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Department Store,Deli / Bodega,Dance Studio,Curling Ice
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Coffee Shop,Pizza Place,Intersection,...,Dog Run,Distribution Center,Deli / Bodega,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Department Store,Cupcake Shop
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Pub,Bakery,...,Beer Store,Spa,Farmers Market,Shoe Store,Yoga Studio,Electronics Store,Distribution Center,Dessert Shop,Chocolate Shop,Performing Arts Venue
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Accessories Store,Furniture / Home Store,Coffee Shop,...,Deli / Bodega,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,0.0,Coffee Shop,Diner,Music Venue,Beer Bar,...,College Auditorium,Creperie,Bank,Mexican Restaurant,Discount Store,Distribution Center,Juice Bar,Italian Restaurant,Fried Chicken Joint,Hobby Shop
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,7.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


now time to plot!

In [53]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[lat, long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lt, lng, poi, cluster in zip(hood_merged['Latitude'], hood_merged['Longitude'], hood_merged['Neighborhood'], hood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lt, lng],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now lets examine each cluster and look at the top 2 places for each 

In [52]:

for j in range(kclusters+1):
    print('\nLooking at CLUSTER ', j , ' : ' )
    print(hood_merged.loc[hood_merged['Cluster Labels'] == j, hood_merged.columns[[1] + list(range(6, 8))]])
    



Looking at CLUSTER  0  : 
              Borough       1st Most Common Venue 2nd Most Common Venue
1          North York                Hockey Arena           Coffee Shop
2    Downtown Toronto                 Coffee Shop                  Park
3          North York              Clothing Store     Accessories Store
4    Downtown Toronto                 Coffee Shop                 Diner
6         Scarborough  Construction & Landscaping  Fast Food Restaurant
..                ...                         ...                   ...
96   Downtown Toronto                 Coffee Shop                  Café
97   Downtown Toronto                 Coffee Shop                  Café
99   Downtown Toronto                 Coffee Shop   Japanese Restaurant
100      East Toronto                 Yoga Studio         Auto Workshop
102         Etobicoke           Convenience Store           Social Club

[85 rows x 3 columns]

Looking at CLUSTER  1  : 
   Borough 1st Most Common Venue 2nd Most Common Venue
64  